In [ ]:
# STEP 1: Install Required Packages
!pip install transformers sentence-transformers gradio torch pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 87.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 67.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 38.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [ ]:
# STEP 2: Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# STEP 3: Load Fine-Tuned FLAN-T5 from Drive
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch
import re

model_path = "/content/drive/MyDrive/flan-t5-therapy-finetuned"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSeq2SeqLM.from_pretrained(model_path)

In [ ]:
# STEP 4: Load Sentence-BERT for Retrieval
from sentence_transformers import SentenceTransformer, util
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# STEP 5: Load FAQ and Symptom Embeddings
import pickle

faq_path = "/content/drive/MyDrive/faq_embeddings.pkl"
symptom_path = "/content/drive/MyDrive/symptom_embeddings.pkl"

with open(faq_path, "rb") as f:
    faq_data = pickle.load(f)

with open(symptom_path, "rb") as f:
    symptom_data = pickle.load(f)

faq_texts = faq_data.get("questions", list(faq_data.keys()))
faq_answers = faq_data.get("answers", list(faq_data.values()))
faq_embeddings = embedding_model.encode(faq_texts, convert_to_tensor=True)

symptom_texts = symptom_data.get("symptoms", list(symptom_data.keys()))
symptom_answers = symptom_data.get("responses", list(symptom_data.values()))
symptom_embeddings = embedding_model.encode(symptom_texts, convert_to_tensor=True)

In [ ]:
# STEP 6: Helpers – Clean Names & Remove Duplicate Phrases
def clean_name(text):
    text = re.sub(r"\b(?:Charlie|Alex|John|Sarah|Emily|David)\b", "you", text)
    text = re.sub(r"Hi,? you", "Hi", text)
    return text.replace("you,", "you").strip()

def remove_duplicate_lines(text):
    lines = text.split(". ")
    seen = set()
    result = []
    for line in lines:
        if line not in seen:
            result.append(line)
            seen.add(line)
    return ". ".join(result)

In [ ]:
# STEP 7: Generation Function

def generate_therapist_reply(user_input):
    prompt = f"Respond like a therapist: {user_input}"
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, padding=True)
    outputs = model.generate(
        **inputs,
        max_new_tokens=100,
        do_sample=True,
        top_k=50,
        top_p=0.95,
        temperature=0.8
    )
    raw = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return clean_name(remove_duplicate_lines(raw))

In [ ]:
# STEP 8: Hybrid Response with Memory
last_symptom = None

def hybrid_response(user_input):
    global last_symptom
    query_embedding = embedding_model.encode(user_input, convert_to_tensor=True)

    symptom_scores = util.pytorch_cos_sim(query_embedding, symptom_embeddings)[0]
    faq_scores = util.pytorch_cos_sim(query_embedding, faq_embeddings)[0]

    max_symptom_score = torch.max(symptom_scores).item()
    max_faq_score = torch.max(faq_scores).item()

    if max_symptom_score > 0.65:
        idx = torch.argmax(symptom_scores).item()
        last_symptom = symptom_texts[idx]
        return f"[Symptom Advice]\n{symptom_answers[idx]}"
    elif max_faq_score > 0.65:
        idx = torch.argmax(faq_scores).item()
        return f"[FAQ Answer]\n{faq_answers[idx]}"
    else:
        if "illness" in user_input.lower() and last_symptom:
            return f"Earlier you mentioned '{last_symptom}'. Would you like to talk more about it?"
        return f"[Therapist Generated]\n{generate_therapist_reply(user_input)}"


In [ ]:
# Gradio UI
import gradio as gr
chat_history = []

def chat_interface(user_input):
    global chat_history
    response = hybrid_response(user_input)
    chat_history.append((f"👤 You: {user_input}", f"🤖 Bot: {response}"))
    return chat_history, ""

def clear_chat():
    global chat_history
    chat_history = []
    return [], ""

with gr.Blocks(css="""
#header1, #header2 { text-align: center; }
body { background-color: #0f172a; color: white; font-family: 'Segoe UI', sans-serif; }
.gr-button { border-radius: 8px; font-weight: 600; }
.gr-textbox textarea { background-color: #1e293b; color: white; border: 1px solid #334155; }
.gr-chatbot { background-color: #1e293b; border-radius: 10px; padding: 1rem; color: white; }
.gr-markdown h2 { text-align: center; color: #f472b6; }
""") as demo:

    gr.Markdown("## 🧠 TherapyBot++", elem_id="header1")
    gr.Markdown("Thoughtfully designed to remember what matters to you — and to be there when it matters the most.", elem_id="header2")

    with gr.Row():
        with gr.Column(scale=5):
            gr.Markdown("#### 📝 Your Message")
            msg = gr.Textbox(placeholder="How are you feeling?", lines=4, label=None)
            with gr.Row():
                submit = gr.Button("Submit", variant="primary")
                clear = gr.Button("Clear")
        with gr.Column(scale=8):
            gr.Markdown("#### 💬 Full Chat Conversation")
            chatbot = gr.Chatbot()

    submit.click(chat_interface, inputs=[msg], outputs=[chatbot, msg])
    clear.click(clear_chat, outputs=[chatbot, msg])

demo.launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://da82dbe64197121973.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
